# Aadhaar Data Preprocessing Pipeline

**Objective:** Clean and preprocess the combined Aadhaar datasets (Enrolment, Demographic Updates, Biometric Updates) for analysis.

**Datasets:**
- `api_data_aadhar_enrolment_combined.csv` (~46 MB)
- `api_data_aadhar_demographic_combined.csv` (~92 MB)
- `api_data_aadhar_biometric_combined.csv` (~82 MB)

**Steps:**
1. Load and inspect data structure
2. Handle missing values
3. Fix data types and formats
4. Identify and handle anomalies
5. Create clean datasets for analysis

## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Combined Datasets

We'll load all three combined datasets and get initial overview of their structure.

In [2]:
# Define paths using relative paths (works from notebooks/ directory)
base_path = Path("../data/processed")

# Load datasets
print("Loading datasets...")
df_enrolment = pd.read_csv(base_path / "api_data_aadhar_enrolment_combined.csv")
df_demographic = pd.read_csv(base_path / "api_data_aadhar_demographic_combined.csv")
df_biometric = pd.read_csv(base_path / "api_data_aadhar_biometric_combined.csv")

print(f"\n✓ Enrolment Dataset: {df_enrolment.shape[0]:,} rows × {df_enrolment.shape[1]} columns")
print(f"✓ Demographic Dataset: {df_demographic.shape[0]:,} rows × {df_demographic.shape[1]} columns")
print(f"✓ Biometric Dataset: {df_biometric.shape[0]:,} rows × {df_biometric.shape[1]} columns")

Loading datasets...

✓ Enrolment Dataset: 1,006,029 rows × 7 columns
✓ Demographic Dataset: 2,071,700 rows × 6 columns
✓ Biometric Dataset: 1,861,108 rows × 6 columns


## 3. Initial Data Inspection

Examine column names, data types, and preview samples from each dataset.

In [3]:
# Function to create detailed dataset summary
def inspect_dataset(df, name):
    print(f"\n{'='*80}")
    print(f"Dataset: {name}")
    print(f"{'='*80}")
    print(f"\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nData Types:\n{df.dtypes}")
    print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"\nFirst 3 rows:")
    display(df.head(3))
    
# Inspect all datasets
inspect_dataset(df_enrolment, "Enrolment")
inspect_dataset(df_demographic, "Demographic Updates")
inspect_dataset(df_biometric, "Biometric Updates")


Dataset: Enrolment

Shape: 1,006,029 rows × 7 columns

Columns: ['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater']

Data Types:
date              object
state             object
district          object
pincode            int64
age_0_5            int64
age_5_17           int64
age_18_greater     int64
dtype: object

Memory Usage: 199.12 MB

First 3 rows:


,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,02-03-2025,Meghalaya,East Khasi Hills,793121,11,61,37
1,09-03-2025,Karnataka,Bengaluru Urban,560043,14,33,39
2,09-03-2025,Uttar Pradesh,Kanpur Nagar,208001,29,82,12



Dataset: Demographic Updates

Shape: 2,071,700 rows × 6 columns

Columns: ['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17_']

Data Types:
date             object
state            object
district         object
pincode           int64
demo_age_5_17     int64
demo_age_17_      int64
dtype: object

Memory Usage: 394.80 MB

First 3 rows:


,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,Uttar Pradesh,Gorakhpur,273213,49,529
1,01-03-2025,Andhra Pradesh,Chittoor,517132,22,375
2,01-03-2025,Gujarat,Rajkot,360006,65,765



Dataset: Biometric Updates

Shape: 1,861,108 rows × 6 columns

Columns: ['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_']

Data Types:
date            object
state           object
district        object
pincode          int64
bio_age_5_17     int64
bio_age_17_      int64
dtype: object

Memory Usage: 354.45 MB

First 3 rows:


,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,Haryana,Mahendragarh,123029,280,577
1,01-03-2025,Bihar,Madhepura,852121,144,369
2,01-03-2025,Jammu and Kashmir,Punch,185101,643,1091


## 4. Missing Value Analysis

Identify and quantify missing values in each dataset.

In [4]:
# Function to analyze missing values
def analyze_missing(df, name):
    print(f"\n{'='*80}")
    print(f"Missing Value Analysis: {name}")
    print(f"{'='*80}")
    
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing_Count': missing.values,
        'Missing_Percentage': missing_pct.values
    })
    missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
    
    if len(missing_df) > 0:
        print(f"\nColumns with missing values:\n")
        display(missing_df)
        
        # Visualize
        if len(missing_df) <= 20:
            plt.figure(figsize=(10, 6))
            plt.barh(missing_df['Column'], missing_df['Missing_Percentage'])
            plt.xlabel('Missing Percentage (%)')
            plt.title(f'Missing Values - {name}')
            plt.tight_layout()
            plt.show()
    else:
        print("\n✓ No missing values found!")
    
    return missing_df

# Analyze missing values
missing_enrolment = analyze_missing(df_enrolment, "Enrolment")
missing_demographic = analyze_missing(df_demographic, "Demographic Updates")
missing_biometric = analyze_missing(df_biometric, "Biometric Updates")


Missing Value Analysis: Enrolment

✓ No missing values found!

Missing Value Analysis: Demographic Updates

✓ No missing values found!

Missing Value Analysis: Biometric Updates

✓ No missing values found!


## 5. Data Type Validation & Conversion

Check and convert data types appropriately (dates, numerics, categoricals).

In [5]:
# Function to identify and convert date columns
def convert_date_columns(df, name):
    print(f"\n{'='*60}")
    print(f"Converting date columns for: {name}")
    print(f"{'='*60}")
    
    # Common date column patterns
    date_keywords = ['date', 'Date', 'DATE', 'time', 'Time', 'TIME']
    
    for col in df.columns:
        if any(keyword in col for keyword in date_keywords):
            try:
                # Try to convert to datetime
                df[col] = pd.to_datetime(df[col], errors='coerce')
                print(f"✓ Converted '{col}' to datetime")
            except Exception as e:
                print(f"✗ Could not convert '{col}': {e}")
    
    return df

# Convert date columns
df_enrolment = convert_date_columns(df_enrolment, "Enrolment")
df_demographic = convert_date_columns(df_demographic, "Demographic Updates")
df_biometric = convert_date_columns(df_biometric, "Biometric Updates")


Converting date columns for: Enrolment
✓ Converted 'date' to datetime

Converting date columns for: Demographic Updates
✓ Converted 'date' to datetime

Converting date columns for: Biometric Updates
✓ Converted 'date' to datetime


## 6. Statistical Summary & Anomaly Detection

Generate statistical summaries and identify potential outliers or anomalies.

In [6]:
# Generate statistical summaries
def statistical_summary(df, name):
    print(f"\n{'='*80}")
    print(f"Statistical Summary: {name}")
    print(f"{'='*80}")
    
    # Numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\nNumeric columns summary:")
        display(df[numeric_cols].describe())
    
    # Categorical/Object columns
    object_cols = df.select_dtypes(include=['object']).columns
    if len(object_cols) > 0:
        print(f"\nCategorical columns:")
        for col in object_cols[:10]:  # Show first 10
            print(f"\n{col}: {df[col].nunique()} unique values")
            print(df[col].value_counts().head(5))

# Generate summaries
statistical_summary(df_enrolment, "Enrolment")
statistical_summary(df_demographic, "Demographic Updates")
statistical_summary(df_biometric, "Biometric Updates")


Statistical Summary: Enrolment

Numeric columns summary:


,pincode,age_0_5,age_5_17,age_18_greater
count,1.006029e+06,1.006029e+06,1.006029e+06,1.006029e+06
mean,5.186415e+05,3.525709e+00,1.710074e+00,1.673441e-01
std,2.056360e+05,1.753851e+01,1.436963e+01,3.220525e+00
min,1.000000e+05,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.636410e+05,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.174170e+05,2.000000e+00,0.000000e+00,0.000000e+00
75%,7.001040e+05,3.000000e+00,1.000000e+00,0.000000e+00
max,8.554560e+05,2.688000e+03,1.812000e+03,8.550000e+02



Categorical columns:

state: 55 unique values
state
Uttar Pradesh    110369
Tamil Nadu        92552
Maharashtra       77191
West Bengal       76519
Karnataka         70198
Name: count, dtype: int64

district: 985 unique values
district
Pune                 6663
North 24 Parganas    6488
Barddhaman           5362
Bengaluru            5305
Hyderabad            4984
Name: count, dtype: int64

Statistical Summary: Demographic Updates

Numeric columns summary:


,pincode,demo_age_5_17,demo_age_17_
count,2.071700e+06,2.071700e+06,2.071700e+06
mean,5.278318e+05,2.347552e+00,2.144701e+01
std,1.972933e+05,1.490355e+01,1.252498e+02
min,1.000000e+05,0.000000e+00,0.000000e+00
25%,3.964690e+05,0.000000e+00,2.000000e+00
50%,5.243220e+05,1.000000e+00,6.000000e+00
75%,6.955070e+05,2.000000e+00,1.500000e+01
max,8.554560e+05,2.690000e+03,1.616600e+04



Categorical columns:

state: 65 unique values
state
Andhra Pradesh    207687
Tamil Nadu        196857
West Bengal       168623
Uttar Pradesh     167889
Maharashtra       162242
Name: count, dtype: int64

district: 983 unique values
district
North 24 Parganas    12994
Pune                 12450
Barddhaman           12349
East Godavari        12186
Thrissur             12097
Name: count, dtype: int64

Statistical Summary: Biometric Updates

Numeric columns summary:


,pincode,bio_age_5_17,bio_age_17_
count,1.861108e+06,1.861108e+06,1.861108e+06
mean,5.217612e+05,1.839058e+01,1.909413e+01
std,1.981627e+05,8.370421e+01,8.806502e+01
min,1.100010e+05,0.000000e+00,0.000000e+00
25%,3.911750e+05,1.000000e+00,1.000000e+00
50%,5.224010e+05,3.000000e+00,4.000000e+00
75%,6.866362e+05,1.100000e+01,1.000000e+01
max,8.554560e+05,8.002000e+03,7.625000e+03



Categorical columns:

state: 57 unique values
state
Tamil Nadu        184568
Andhra Pradesh    172034
Uttar Pradesh     155242
Maharashtra       151104
Karnataka         141227
Name: count, dtype: int64

district: 974 unique values
district
Pune                 11586
Thrissur             11165
East Godavari        10647
North 24 Parganas    10595
Barddhaman           10545
Name: count, dtype: int64


## 7. Data Quality Report

Create a comprehensive data quality report for all datasets.

In [7]:
# Comprehensive data quality report
def data_quality_report(df, name):
    report = {
        'Dataset': name,
        'Total_Rows': df.shape[0],
        'Total_Columns': df.shape[1],
        'Duplicate_Rows': df.duplicated().sum(),
        'Total_Missing_Values': df.isnull().sum().sum(),
        'Missing_Percentage': (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100,
        'Memory_MB': df.memory_usage(deep=True).sum() / 1024**2,
        'Numeric_Columns': len(df.select_dtypes(include=[np.number]).columns),
        'Object_Columns': len(df.select_dtypes(include=['object']).columns),
        'Datetime_Columns': len(df.select_dtypes(include=['datetime64']).columns)
    }
    return report

# Generate quality reports
reports = []
reports.append(data_quality_report(df_enrolment, "Enrolment"))
reports.append(data_quality_report(df_demographic, "Demographic Updates"))
reports.append(data_quality_report(df_biometric, "Biometric Updates"))

quality_df = pd.DataFrame(reports)
print("\n" + "="*100)
print("DATA QUALITY REPORT SUMMARY")
print("="*100)
display(quality_df)


DATA QUALITY REPORT SUMMARY


,Dataset,Total_Rows,Total_Columns,Duplicate_Rows,Total_Missing_Values,Missing_Percentage,Memory_MB,Numeric_Columns,Object_Columns,Datetime_Columns
0,Enrolment,1006029,7,385118,682238,9.687849,150.201696,4,2,1
1,Demographic Updates,2071700,6,823227,1187968,9.557111,294.077966,3,2,1
2,Biometric Updates,1861108,6,331623,944100,8.454641,263.956126,3,2,1


## 8. Data Cleaning Pipeline

Apply cleaning operations based on the quality assessment.

In [8]:
# Clean data function
def clean_dataset(df, name):
    print(f"\n{'='*80}")
    print(f"Cleaning: {name}")
    print(f"{'='*80}")
    
    df_clean = df.copy()
    initial_rows = len(df_clean)
    
    # 1. Remove duplicate rows
    duplicates = df_clean.duplicated().sum()
    if duplicates > 0:
        df_clean = df_clean.drop_duplicates()
        print(f"✓ Removed {duplicates:,} duplicate rows")
    
    # 2. Handle missing values (strategy depends on column type and missing %)
    for col in df_clean.columns:
        missing_pct = (df_clean[col].isnull().sum() / len(df_clean)) * 100
        
        if missing_pct > 0:
            if missing_pct > 50:
                # Drop columns with >50% missing
                print(f"✗ Dropping column '{col}' ({missing_pct:.1f}% missing)")
                df_clean = df_clean.drop(columns=[col])
            elif df_clean[col].dtype in [np.number, 'int64', 'float64']:
                # Fill numeric with median
                df_clean[col].fillna(df_clean[col].median(), inplace=True)
                print(f"✓ Filled numeric '{col}' with median ({missing_pct:.1f}% missing)")
            else:
                # Fill categorical with mode or 'Unknown'
                if df_clean[col].mode().shape[0] > 0:
                    df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
                else:
                    df_clean[col].fillna('Unknown', inplace=True)
                print(f"✓ Filled categorical '{col}' with mode/Unknown ({missing_pct:.1f}% missing)")
    
    # 3. Remove rows with any remaining nulls (if critical)
    remaining_nulls = df_clean.isnull().sum().sum()
    if remaining_nulls > 0:
        df_clean = df_clean.dropna()
        print(f"✓ Dropped {initial_rows - len(df_clean):,} rows with remaining nulls")
    
    final_rows = len(df_clean)
    print(f"\n✓ Final: {final_rows:,} rows ({initial_rows - final_rows:,} removed, {((initial_rows - final_rows)/initial_rows)*100:.2f}% reduction)")
    
    return df_clean

# Clean all datasets
df_enrolment_clean = clean_dataset(df_enrolment, "Enrolment")
df_demographic_clean = clean_dataset(df_demographic, "Demographic Updates")
df_biometric_clean = clean_dataset(df_biometric, "Biometric Updates")


Cleaning: Enrolment
✓ Removed 385,118 duplicate rows
✓ Filled categorical 'date' with mode/Unknown (48.5% missing)

✓ Final: 620,911 rows (385,118 removed, 38.28% reduction)

Cleaning: Demographic Updates
✓ Removed 823,227 duplicate rows
✓ Filled categorical 'date' with mode/Unknown (45.8% missing)

✓ Final: 1,248,473 rows (823,227 removed, 39.74% reduction)

Cleaning: Biometric Updates
✓ Removed 331,623 duplicate rows
✓ Filled categorical 'date' with mode/Unknown (43.0% missing)

✓ Final: 1,529,485 rows (331,623 removed, 17.82% reduction)


## 9. Save Cleaned Datasets

Export preprocessed datasets for analysis.

In [9]:
# Create output directory using relative path
output_path = Path("../data/processed")
output_path.mkdir(parents=True, exist_ok=True)

# Save cleaned datasets
print("Saving cleaned datasets...")
df_enrolment_clean.to_csv(output_path / "enrolment_clean.csv", index=False)
print(f"✓ Saved: enrolment_clean.csv ({len(df_enrolment_clean):,} rows)")

df_demographic_clean.to_csv(output_path / "demographic_clean.csv", index=False)
print(f"✓ Saved: demographic_clean.csv ({len(df_demographic_clean):,} rows)")

df_biometric_clean.to_csv(output_path / "biometric_clean.csv", index=False)
print(f"✓ Saved: biometric_clean.csv ({len(df_biometric_clean):,} rows)")

print(f"\n✓ All cleaned datasets saved to: {output_path.absolute()}")

Saving cleaned datasets...
✓ Saved: enrolment_clean.csv (620,911 rows)
✓ Saved: demographic_clean.csv (1,248,473 rows)
✓ Saved: biometric_clean.csv (1,529,485 rows)

✓ All cleaned datasets saved to: c:\Users\user\OneDrive\Desktop\CODE\UIDAI\notebooks\..\data\processed


## 10. Preprocessing Summary

Final summary of the preprocessing pipeline.

In [10]:
# Create comprehensive summary
summary = pd.DataFrame({
    'Dataset': ['Enrolment', 'Demographic', 'Biometric'],
    'Original_Rows': [len(df_enrolment), len(df_demographic), len(df_biometric)],
    'Cleaned_Rows': [len(df_enrolment_clean), len(df_demographic_clean), len(df_biometric_clean)],
    'Rows_Removed': [
        len(df_enrolment) - len(df_enrolment_clean),
        len(df_demographic) - len(df_demographic_clean),
        len(df_biometric) - len(df_biometric_clean)
    ],
    'Retention_Rate_%': [
        (len(df_enrolment_clean) / len(df_enrolment)) * 100,
        (len(df_demographic_clean) / len(df_demographic)) * 100,
        (len(df_biometric_clean) / len(df_biometric)) * 100
    ]
})

print("\n" + "="*100)
print("PREPROCESSING SUMMARY")
print("="*100)
display(summary)

print("\n✓ Data preprocessing completed successfully!")
print("✓ Cleaned datasets are ready for analysis")
print(f"✓ Location: {output_path}")


PREPROCESSING SUMMARY


,Dataset,Original_Rows,Cleaned_Rows,Rows_Removed,Retention_Rate_%
0,Enrolment,1006029,620911,385118,61.718996
1,Demographic,2071700,1248473,823227,60.263214
2,Biometric,1861108,1529485,331623,82.181421



✓ Data preprocessing completed successfully!
✓ Cleaned datasets are ready for analysis
✓ Location: ..\data\processed
